In [1]:
import re
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_all = pd.read_pickle('./data/df_all_lemma.pkl')
print(len(df_all))
df_all.head(1)

2467


,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,transcript
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,it been great hasnt it. ive been blown away by...


In [18]:
docs_all = list(df_all['transcript'])

from nltk.corpus import stopwords

stops_standard = stopwords.words('english')
stops_custom = ['shes','youll','ill','yeah','th','yes','oh',
                'ok','okay','might','ha','mr','bg','ms',
                'mrs','ca','em','da','ted']+[re.sub('[^A-Za-z ]+', '', w) for w in stops_standard]
stop_list = list(set(stops_standard + stops_custom))

print(stop_list)

['da', 'each', "wasn't", 'our', 'which', 'few', 'for', 'down', "should've", 'ain', 'youll', 'youd', 'both', "mustn't", 'we', 'had', 'where', 'their', 'until', 'oh', 'it', 'his', 'than', 'hers', 'has', 'too', 'an', "haven't", 'should', 'i', 'shouldve', "you'll", 'does', 'll', "you're", 'over', 'your', "mightn't", 'havent', 'weren', 'again', 'no', 'ted', 'themselves', 'here', 'this', 'okay', 'own', 'mustnt', "isn't", 'that', 'me', 'he', 'be', 'herself', 'ours', 'haven', "didn't", 'might', 'being', 'into', 'wont', 'have', 'mightn', 'needn', 'th', 'bg', 'isnt', 'my', 'only', 'on', 'thatll', "you've", 'did', 'wouldnt', 'wasnt', 'himself', "it's", 'youre', 'out', 'up', 'couldnt', 're', 'by', 'don', 'they', 'but', 'under', 'yourself', 'from', 'mr', 'mrs', 'arent', 'if', 'couldn', "couldn't", 'didnt', 'same', 'those', 'in', 'shes', 'theirs', "shouldn't", 'not', 'why', 'now', 'shant', 'd', 'am', 'of', 'through', "she's", "hasn't", 'hadnt', 'to', 'hadn', 'aren', 'm', "don't", 'o', "doesn't", 'he

In [34]:
docs = docs_all

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

n_samples = 2000
n_features = 2000
n_components = 50
n_top_words = 10
n_gram = 1
alpha = 0.1
stop_choice= stop_list

max_df = 300
min_df = 10
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df,
                                   ngram_range=(n_gram,n_gram),
                                   max_features=n_features,
                                   stop_words=stop_choice)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(docs)
print("done in %0.3fs." % (time() - t0))

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=alpha, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 3.070s.
Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=2000...
done in 3.736s.

Topics in NMF model (Frobenius norm):
Topic #0: sleep sensor alzheimers clock teenager bed vessel waste meeting virtual
Topic #1: patient surgery physician medicine treatment healthcare clinic nurse pain surgeon
Topic #2: climate nuclear carbon emission ice fuel co solar coal gas
Topic #3: robot leg ai robotic sensor intelligence motor ant artificial drone
Topic #4: farmer farm fish feed bread agriculture seed meat organic pig
Topic #5: universe particle theory quantum physic galaxy atom bang string physicist
Topic #6: zoom facing exploring exposed express expression extent extra extract extraordinary
Topic #7: dna gene genome genetic bacteria chromosome molecule sequence organism code
Topic #8: cancer tumor breast blood treatment protein lung molecule medicine gene
Topic #9: ocean fish sea coral shark reef whale marin

In [30]:
X = nmf.fit_transform(tfidf)


In [32]:
X[1000].argmax()

1

### Find related talks

In [14]:
import sklearn.neighbors.kd_tree as kdtree

In [33]:
tree = kdtree.KDTree(X, leaf_size=10) 
N = 1000
dist, ind = tree.query([X[N]], k=20)  
for i in ind[0]:
    print(i,'\t',df_all.iloc[i].title)
#     print(i,'\t',df_all.iloc[i].views)
#     print(df_all.iloc[i].tags)

1000 	 A prosthetic arm that "feels"
2264 	 Could a drug prevent depression and PTSD?
686 	 A second opinion on developmental disorders
901 	 The mystery of chronic pain
1045 	 A prosthetic eye to treat blindness
1343 	 A universal translator for surgeons
602 	 How brains learn to see
615 	 Take health care off the mainframe
1117 	 Let's talk about dying
506 	 What hallucination reveals about our minds
443 	 Are we in control of our own decisions?
399 	 Our buggy moral code
1264 	 What doctors don't know about the drugs they prescribe
1200 	 A tale of mental illness -- from the inside
1752 	 A neural portrait of the human mind
788 	 How I taught rats to sniff out land mines
1048 	 There are no scraps of men
2076 	 Simple hacks for life with Parkinson's
2153 	 What can we learn from shortcuts?
1334 	 Ethical riddles in HIV research


In [25]:
df_related = pd.DataFrame(eval(df_all.iloc[N]['related_talks']))
for k in range(len(df_related)):
    print(list(df_all[df_all['title']==df_related.iloc[k]['title']].title))
    print(df_all[df_all['title']==df_related.iloc[k]['title']].views)
#     print(list(df_all[df_all['title']==df_related.iloc[k]['title']].tags))

['My 12 pairs of legs']
395    3472415
Name: views, dtype: int64
['Luke, a new prosthetic arm for soldiers']
134    1575699
Name: views, dtype: int64
['The potential of regenerative medicine']
118    1427723
Name: views, dtype: int64
['The mystery of chronic pain']
901    1778606
Name: views, dtype: int64
['The emotion behind invention']
630    523422
Name: views, dtype: int64
['3 clues to understanding your brain']
151    3930186
Name: views, dtype: int64


In [28]:
print(docs_all[395])

they only learn to be frightened of those difference when an adult influence them to behave that way and maybe censor that natural curiosity or you know rein in the questionasking in the hope of them being polite little kid. so i just pictured a first grade teacher out in the lobby with these unruly kid saying now whatever you do dont stare at her leg. but of course thats the point. thats why i wa there i wanted to invite them to look and explore. so i made a deal with the adult that the kid could come in without any adult for two minute on their own. the door open the kid descend on this table of leg and they are poking and prodding and theyre wiggling toe and theyre trying to put their full weight on the sprinting leg to see what happens with that. and i said kid really quickly i woke up this morning i decided i wanted to be able to jump over a house nothing too big two or three story but if you could think of any animal any superhero any cartoon character anything you can dream up r

### Clustering using KMeans

In [509]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.metrics import silhouette_score
from sklearn import cluster

n_clusters = 50

# ward = cluster.AgglomerativeClustering(
#     n_clusters=n_clusters, linkage='ward',
#     connectivity=connectivity)
spectral = cluster.SpectralClustering(
    n_clusters=n_clusters, eigen_solver='arpack',
    affinity="nearest_neighbors")
Xp = spectral.fit_predict(X)
# dbscan = cluster.DBSCAN(eps=params['eps'])

# km = KMeans(n_clusters=50)
# Xt = km.fit_transform(X)

# mu_docs = km.cluster_centers_
# plt.plot(mu_docs.transpose())

# silhouette_score(X, km.labels_, metric='euclidean')

# sil_scores = []
# iner_scores = []
# ks = range(2,20)
# for k in ks:
#     km = KMeans(n_clusters=k)
#     km.fit(X)
#     sil_scores.append(silhouette_score(X, km.labels_, metric='euclidean'))
#     iner_scores.append(km.inertia_)
    
# plt.plot(ks,sil_scores)
# plt.plot(ks,iner_scores)

In [510]:
Xp.

array([18, 29, 18, ..., 28, 18, 10], dtype=int32)

In [505]:
tree = kdtree.KDTree(Xt, leaf_size=10) 
# N = 1346
dist, ind = tree.query([Xt[N]], k=10)  
for i in ind[0]:
    print(i,'\t',df_all.iloc[i].title)
#     print(i,'\t',df_all.iloc[i].views)
#     print(df_all.iloc[i].tags)

1599 	 Love -- you're doing it wrong
2343 	 3 ways to plan for the (very) long term
2085 	 How to get back to work after a career break
1395 	 What makes us feel good about our work?
1135 	 How I beat a patent troll
1872 	 How equal do we want the world to be? You'd be surprised
1821 	 The danger of hiding who you are
1870 	 5 ways to kill your dreams
1566 	 The $80 prosthetic knee that's changing lives
609 	 Plug into your hard-wired happiness


In [506]:
df_all.iloc[2343].transcript

'and theyd say great. and ive been seeing that time horizon get shorter and shorter and shorter so much so that i met with a ceo two month ago and i said we started our initial conversation. he go i love what you do. i want to talk about the next six month. we have a lot of problem that we are facing. these are civilizationalscale problem. the issue though is we cant solve them using the mental model that we use right now to try and solve these problem. yes a lot of great technical work is being done but there is a problem that we need to solve for a priori before if we want to really move the needle on those big problem. shorttermism. right. there no march. there no bracelet. there no petition that you can sign to be against shorttermism. i tried to put one up and no one signed. it wa weird. but it prevents u from doing so much. shorttermism for many reason ha pervaded every nook and cranny of our reality. i just want you to take a second and just think about an issue that youre think